In [1]:
!git clone https://github.com/ykamoji/Modality-Aware-MMLM-Retriever.git

Cloning into 'Modality-Aware-MMLM-Retriever'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 145 (delta 68), reused 112 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (145/145), 643.96 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [1]:
%cd Modality-Aware-MMLM-Retriever

/content/Modality-Aware-MMLM-Retriever


In [10]:
!pip3 install datasets
!pip3 install faiss-cpu faiss-gpu pytrec_eval tensorboard

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Update DATASET_PATH in cofig.yaml
!python3 datasetUtils/load_dataset.py

README.md: 100% 7.87k/7.87k [00:00<00:00, 38.9MB/s]
mbeir_fashion200k_train.jsonl: 100% 7.43M/7.43M [00:00<00:00, 25.1MB/s]
Generating train split: 30000 examples [00:00, 226011.37 examples/s]
Training data:
Dataset({
    features: ['qid', 'query_txt', 'query_img_path', 'query_modality', 'query_src_content', 'pos_cand_list', 'neg_cand_list', 'task_id'],
    num_rows: 30000
})
mbeir_fashion200k_task0_val.jsonl: 100% 415k/415k [00:00<00:00, 33.0MB/s]
Generating val split: 1746 examples [00:00, 312652.30 examples/s]
mbeir_fashion200k_task3_val.jsonl: 100% 1.26M/1.26M [00:00<00:00, 10.6MB/s]
Generating val split: 4860 examples [00:00, 447358.06 examples/s]
Validation data:
Dataset({
    features: ['qid', 'query_txt', 'query_img_path', 'query_modality', 'query_src_content', 'pos_cand_list', 'neg_cand_list', 'task_id'],
    num_rows: 6606
})
mbeir_union_val_cand_pool.jsonl: 100% 48.0M/48.0M [00:01<00:00, 38.7MB/s]
Generating cand_pool split: 136733 examples [00:00, 380658.54 examples/s]
mbei

In [ ]:
from utils.argUtils import CustomObject, get_yaml_loader
import yaml
import json

configPath = 'config.yaml'

In [3]:
from fineTuning import train
with open(configPath, 'r') as file:
    config = yaml.load(file, get_yaml_loader())

x = json.dumps(config)
Args = json.loads(x, object_hook=lambda d: CustomObject(**d))

if Args.FineTuning.Action:
  train(Args)

Training data:
Dataset({
    features: ['qid', 'query_txt', 'query_img_path', 'query_modality', 'query_src_content', 'pos_cand_list', 'neg_cand_list', 'task_id'],
    num_rows: 100
})
Candidate data:
Dataset({
    features: ['did', 'txt', 'img_path', 'modality', 'src_content'],
    num_rows: 69557
})
Validation data:
Dataset({
    features: ['qid', 'query_txt', 'query_img_path', 'query_modality', 'query_src_content', 'pos_cand_list', 'neg_cand_list', 'task_id'],
    num_rows: 100
})
Candidate data:
Dataset({
    features: ['did', 'txt', 'img_path', 'modality', 'src_content'],
    num_rows: 69557
})


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy,Modality Accuracy
4,2.900700,45.390797,0.000000,0.000000
8,4.833800,41.714134,0.000000,0.020000
12,2.767700,41.633526,0.000000,0.000000
16,2.963000,41.983810,0.000000,0.020000
20,2.859100,41.485611,0.030000,0.500000
24,2.817400,41.619431,0.060000,0.500000


***** train metrics *****
  epoch                    =        1.0
  total_flos               =        0GF
  train_loss               =     3.2523
  train_runtime            = 0:10:18.34
  train_samples_per_second =      0.162
  train_steps_per_second   =       0.04


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =       0.04
  eval_loss               =    41.6113
  eval_modality_accuracy  =        0.5
  eval_runtime            = 0:00:05.11
  eval_samples_per_second =     19.534
  eval_steps_per_second   =      0.781


In [5]:
from evaluation import evaluate
with open(configPath, 'r') as file:
    config = yaml.load(file, get_yaml_loader())

x = json.dumps(config)
Args = json.loads(x, object_hook=lambda d: CustomObject(**d))

if Args.Evaluate.Action:
    evaluate(Args)

Validation data:
Dataset({
    features: ['qid', 'query_txt', 'query_img_path', 'query_modality', 'query_src_content', 'pos_cand_list', 'neg_cand_list', 'task_id'],
    num_rows: 100
})
Candidate data:
Dataset({
    features: ['did', 'txt', 'img_path', 'modality', 'src_content'],
    num_rows: 69557
})


***** eval metrics *****
  test_accuracy               =       0.05
  test_loss                   =        0.0
  test_modality_accuracy      =        0.5
  test_model_preparation_time =      0.005
  test_runtime                = 0:00:32.22
  test_samples_per_second     =      3.103
  test_steps_per_second       =      0.124

query embedding size: (100, 512), ids: (100,)
query embeddings saved to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/embeddings/

candidate embedding size: (253, 512), ids: (253,)
candidate embeddings saved to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/embeddings/


In [7]:
from retriever.indexer import create_index
from retriever.search import run_retrieval
from retriever.analyse import score_results
with open(configPath, 'r') as file:
    config = yaml.load(file, get_yaml_loader())

x = json.dumps(config)
Args = json.loads(x, object_hook=lambda d: CustomObject(**d))

if Args.Retrieval.Action:
  create_index(Args)
  run_retrieval(Args)
  score_results(Args)

Creating index.
Successfully indexed 253 documents
Index saved to: Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/MBIER.index


Batched Index Search: 100%|██████████| 13/13 [00:00<00:00, 1634.32it/s]


Completed search. Saving qrel run file.


Scoring: 100it [00:00, 56156.17it/s]


Run file saved to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/run_qrel.json
Running pytrac evaluations.


...

Saved local pytrac evaluations to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/local/Text_2_Image_Fashion200K.json


Task 3 Dataset 1: 100%|██████████| 4860/4860 [00:00<00:00, 2030512.74it/s]


Saved local pytrac evaluations to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/local/Image_2_Text_Fashion200K.json


Task 3 Dataset 9: 100%|██████████| 24994/24994 [00:00<00:00, 3395052.60it/s]

Completed pytrac evaluations.
{
  "Text_2_Image": {
    "Fashion200K": {
      "NDCG@10": 0.0,
      "Recall@5": 0.0,
      "Recall@10": 0.0
    }
  },
  "Image_2_Text": {
    "Fashion200K": {
      "NDCG@10": 11.17896,
      "Recall@5": 14.0,
      "Recall@10": 24.0
    }
  }
}
Saved pytrac evaluations to Results/Evaluation/clip-vit-base-patch16/MBIER/run_0/results.json
